# Manual pipeline for DCM download, decompress and classification of views

In [1]:
import os
import sys
import pandas as pd

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir, 'src'))

import d00_utils.db_utils as db
from d02_intermediate.download_dcm import s3_download_decomp_dcm, dcmdir_to_jpgs_for_classification
from d03_classification.predict_views import run_classify, agg_probabilities, predict_views

In [2]:
table_name='instances_w_labels'
train_test_ratio=0.5 
downsample_ratio=0.001
train=False
#to get train_split100, set configs 1.0, 0.05, train=True

dcm_path = os.path.join(os.path.expanduser('~/data'),'01_raw')

#model_path = os.path.join(os.path.expanduser('~/models'), 'view_23_e5_class_11-Mar-2018')
new_mod_direc = '~/dvv/usal_echo/src/d03_classification/echo_deeplearning/models/vgg_13/m1_usal/train/0'
new_mod_name = 'model.ckpt-3200'
model_path = os.path.join(os.path.expanduser(new_mod_direc), new_mod_name)

In [3]:
#dir_name = s3_download_decomp_dcm(dcm_path, table_name, train_test_ratio, downsample_ratio, train)

dir_name = 'test_downsampleby5/'
#dir_name = 'test_split50_downsampleby1000'

In [4]:
dcm_dir = os.path.join(dcm_path, dir_name)
img_dir = os.path.join(os.path.expanduser('~/data'),'02_intermediate', dir_name)

dcmdir_to_jpgs_for_classification(dcm_dir, img_dir)

In [5]:
run_classify(img_dir, model_path, if_exists='replace', feature_dim=1)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Use keras.layers.flatten ins

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
agg_probabilities(if_exists='replace')

In [7]:
predict_views(if_exists='replace')

In [8]:
io_class = db.dbReadWriteClassification()
io_class.list_tables()

['probabilities_frames', 'predictions', 'probabilities_instances', 'test1000_for_m1', 'evaluations', 'ground_truths', 'test1000_for_m2_m3']


In [9]:
df = io_class.get_table('predictions')
print(df.shape)
df.head()

(1036, 8)


,study_id,file_name,model_name,date_run,view23_pred,view4_dev,view4_seg,prediction_id
0,43763,a_43763_1LHFYKEZ,view_23_e5_class_11-Mar-2018,2019-08-14 14:06:23.830776,a4c,a4c,a4c,1
1,45076,a_45076_43IX4IH1,view_23_e5_class_11-Mar-2018,2019-08-14 14:06:23.830776,plax_plax,plax,plax,2
2,46919,a_46919_7SQZPLEJ,view_23_e5_class_11-Mar-2018,2019-08-14 14:06:23.830776,a4c,a4c,a4c,3
3,49268,a_49268_CSTYB214,view_23_e5_class_11-Mar-2018,2019-08-14 14:06:23.830776,plax_plax,plax,plax,4
4,50521,a_50521_EAKXA40T,view_23_e5_class_11-Mar-2018,2019-08-14 14:06:23.830776,a4c,a4c,a4c,5
